In [ ]:
import yaml
import zarr

repo_root = "/Users/miskodzamba/Dropbox/research/gits/spf/"
import sys

sys.path.append(repo_root)  # go to parent dir
from spf.utils import zarr_open_from_lmdb_store


root = "/Volumes/SPFData/missions/april5"
prefix = "wallarrayv3_2024_04_05_22_13_07_nRX2_rx_circle"
zarr_fn = f"{root}/{prefix}.zarr"
yaml_fn = f"{root}/{prefix}.yaml"
z = zarr_open_from_lmdb_store(zarr_fn)

yaml_config = yaml.safe_load(open(yaml_fn, "r"))

In [ ]:
# self.steering_vectors = precompute_steering_vectors(
#     receiver_positions=self.rx_config.rx_pos,
#     carrier_frequency=self.rx_config.lo,
#     spacing=self.nthetas,
# )

from spf.data_collector import rx_config_from_receiver_yaml
from spf.rf import beamformer_given_steering, precompute_steering_vectors
from spf.sdrpluto.sdr_controller import get_avg_phase
import numpy as np
from matplotlib import pyplot as plt
from spf.rf import beamformer_given_steering_nomean

nthetas = 64 + 1

rx_configs = [
    rx_config_from_receiver_yaml(receiver) for receiver in yaml_config["receivers"]
]

steering_vectors = [
    precompute_steering_vectors(
        receiver_positions=rx_config.rx_pos,
        carrier_frequency=rx_config.lo,
        spacing=nthetas,
    )
    for rx_config in rx_configs
]


def load_zarr_to_numpy(x):
    return np.array(x)


session_idx = 10
beam_sds = [
    beamformer_given_steering_nomean(
        steering_vectors=steering_vectors[receiver_idx],
        signal_matrix=load_zarr_to_numpy(
            z.receivers[f"r{receiver_idx}"].signal_matrix[session_idx]
        ),
    )
    for receiver_idx in range(2)
]

# avg_phase_diff = get_avg_phase(signal_matrix)

In [ ]:
from spf.sdrpluto.sdr_controller import get_phase_diff

signal_matrix = load_zarr_to_numpy(z.receivers["r0"].signal_matrix[session_idx])
n = 2 * 4 * 50000
offset = 0
pd = get_phase_diff(signal_matrix[:, offset : offset + n])
v = signal_matrix[0, offset : offset + n]
fig, axs = plt.subplots(2, 1, figsize=(12, 4))
axs[0].scatter(np.arange(n), v.real, alpha=0.1, s=1)
axs[1].scatter(np.arange(n), pd, s=1, alpha=0.1)
from spf.sdrpluto.sdr_controller import circular_mean, circular_stddev, simple_segment


for window in simple_segment(
    pd,
    window_size=2500,
    stride=1250,
    trim=20,
    mean_diff_threshold=0.2,  #
    max_stddev_threshold=0.15,  # just eyeballed this
):
    axs[1].plot(
        [window["start_idx"], window["end_idx"]],
        [window["mean"], window["mean"]],
        color="red",
    )

In [ ]:
from multiprocessing.pool import ThreadPool
from spf.sdrpluto.sdr_controller import (
    get_phase_diff,
    segment_session_star,
    simple_segment,
    segment_session,
)

# z.receivers["r0"].signal_matrix.shape = (sessions, n_rx_per_receiver, samples_per_buffer)
n_sessions, _, samples_per_buffer = z.receivers["r0"].signal_matrix.shape


z = zarr_open_from_lmdb_store(zarr_fn)
from multiprocessing import Pool
import time
import tqdm

start = time.time()
inputs = [(zarr_fn, "r0", idx, 2048, 1024, 20.0, 0.2, 0.15) for idx in range(400)]
with Pool(8) as pool:
    results = list(
        tqdm.tqdm(pool.imap(segment_session_star, inputs), total=len(inputs))
    )
print(time.time() - start)
# start = time.time()
# with ThreadPool(8) as pool:
#     results = pool.starmap(segment_session, [(z, "r0", idx) for idx in range(100)])
# print(time.time() - start)

In [ ]:
z = zarr_open_from_lmdb_store(zarr_fn)
z.store.db.info()

In [ ]:
def test():
    try:
        yield 1
    finally:
        print("WTF")


a = test()
print("OK")
a = None

In [ ]:
zarr_fn
with zarr_open_from_lmdb_store_cm(zarr_fn) as z:
    pass

In [ ]:
from spf.utils import zarr_open_from_lmdb_store_cm


with zarr_open_from_lmdb_store_cm(zarr_fn) as z:
    start = time.time()
    results = list(
        map(
            lambda x: segment_session(zarr_fn, "r0", x, 2048, 1024, 20, 0.2, 0.15),
            range(50),
        )
    )
    print(time.time() - start)

In [ ]:
from numba import njit

repo_root = "/Users/miskodzamba/Dropbox/research/gits/spf/"
import sys
import numpy as np

sys.path.append(repo_root)  # go to parent dir
from spf.utils import random_signal_matrix


@njit
def pi_norm(x):
    return ((x + np.pi) % (2 * np.pi)) - np.pi


@njit
def get_phase_diff(signal_matrix):
    return pi_norm(np.angle(signal_matrix[0]) - np.angle(signal_matrix[1]))


z = random_signal_matrix(n=200)

get_phase_diff(z)

In [ ]:
fake_signal.shape

In [ ]:
z.tree()

In [ ]:
z.receivers.r0.signal_matrix[0].shape